# Building the Sum to Multiplier Skill
This notebook demonstrates how to build a skill which will invoke another from within itself. First we will create a multiplier action, and we will not wrap this into a skill yet. Once we create that action, we will create the sum_to_multiplier skill, which will call upon the multiplier action from within itself in order to retrieve its output

In [ ]:
# Common Setup
%run setup.ipynb

## Building a Cortex Action using iPython Magic
The Cortex Action is a self-contained code modeule that encapsulates a task - in other words, it is a function in computer science terms.  It can be a short running service, long running batch job, or stateful microservice.  For our simple "Hello World" example, we can create our Action using a function and leverage the __%%cortex_action__ iPython cell magic to automatically deploy it to the Cortex Cloud.

This pushes the action to the c12e private registry for your tenant. 

The multiplier action simply takes two numbers and returns their product

In [ ]:
%%cortex_action --name 'class/multiplier-<your_initials>' --function multiplier

from cortex import Message, Cortex

def multiplier(params):
    msg = Message(params)
    num1 = msg.payload.get('num1')
    num2 = msg.payload.get('num2')
    return Message.with_payload({'result': num1*num2}).to_params()

Now we will create the sum_to_multiplier skill. This will call upon the multiplier action, and will need our token in order to be authenticated. It will return the sum of the two input numbers, as well as the output from the multiplier action which is the product of the same two numbers.

In [ ]:
%%cortex_action --name 'class/sum_to_multiplier-<your_initials>' --function car_classifier

from cortex import Message, Cortex

def car_classifier(params):
    
    msg = Message(params)
    
    num1 = msg.payload.get('num1')
    num2 = msg.payload.get('num2')
    
    sum = num1+num2
    
    #We need to enter your token in order to be authneticated by Cortex to call upon the demo/multiplier action
    cortex = Cortex.client(api_endpoint = msg.apiEndpoint, token = msg.token)
    
    #Set our action invocation to be on the multiplier function
    action = cortex.action('class/multiplier-<your_initials>')
    
    #Create the payload and invoke the multiplier function
    rs = action.invoke(Message.with_payload({'num1' : num1, 'num2' : num2}))
    
    #Return the result of the product to this function from multiplier
    result = rs.payload.get('result')
    
    #Set the sum and product to a stirng to ease of readability when this function outputs them
    output = "Sum: " + str(sum) + ", Product: " + str(result)
    
    return Message.with_payload({'Result': output}).to_params()

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()
action = cortex.action('class/sum_to_multiplier-<your_initials>')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we will pass two numbers in, which will first be used by the original skill (sum_to_multiplier) then passed to the multiplier skill and finally will output the result of both.

In [ ]:
#Test payload

from cortex import Message
rs = action.invoke(Message.with_payload({'num1' : 30, 'num2' : 22}))
rs.payload.get('Result')

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Sum to Multiplier" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('class/sum_to_multiplier-<your_initials>').title('Sum to Multiplier-<your_initials>').description('Invoking a skill from within another skill. Takes two numbers and returns the sum and calls another skill to return the product.')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('numbers').title('Numbers').parameter(name='num1', type='number').parameter(name='num2', type='number').all_routing(action, 'output').build()

In the previous step, we referenced an Output called **output**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('output').title('Result').parameter(name='Result', type='string').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to_camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='numbers', message=Message.with_payload({"num1":22, "num2":35}))
rs.payload